In [6]:
import mediapipe as mp
import cv2 
from deepface import DeepFace
import numpy as np
import socket 
import threading
global isConn , result
isConn = 0

In [7]:
def connect():   
    soc = socket.socket()
    global conn,isConn
    soc.bind(('localhost',2000))
    soc.listen(5)
    conn , addr = soc.accept()
    isConn = 1
    print("device connected")

In [8]:
def SendMess(x):
    msg = bytes(f"{x}",'utf-8')
    conn.send(msg)

In [9]:
def emotion_detection():
    connect()
    vid = cv2.VideoCapture(1)
    frameCounter = 0
    while True:
        success, frame = vid.read()
        frameCounter += 1

        if success:
            result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)

            if isinstance(result, list) and len(result) > 0:
                first_result = result[0]

                if 'region' in first_result.keys():
                    faces = first_result['region']

                    if 'x' in faces and 'y' in faces and 'w' in faces and 'h' in faces:
                        x, y, w, h = int(faces['x']), int(faces['y']), int(faces['w']), int(faces['h'])
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                        emotion = first_result['dominant_emotion']
                        text = f"Emotion: {emotion}"
                        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                SendMess(emotion)
            cv2.imshow('Emotion Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()
    
    

In [10]:
thread2 = threading.Thread(target=emotion_detection)
thread2.start()
thread2.join()

device connected
